<a href="https://colab.research.google.com/github/rosselladedo/WorldConsumption/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

### Import

In [ ]:
pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=ef760ecef5184cfc7f47fdd7e5442bd1380591702ddc1faf0a71b4ad7e89dafe
  Stored in directory: /root/.cache/pip/wheels/68/5d/a5/edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [ ]:

# Import pandas and dataFrame
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sb
import re

# Import matplotlib and seaborn for histograms, boxplots, and
# scatterplots
import matplotlib.pyplot as plt
import seaborn

# Sql packages
from pandasql import sqldf
from sqlalchemy import create_engine

# Import packages for regression analysis
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn.metrics import confusion_matrix


In [ ]:
#Leggo dataset
df_original= pd.read_csv("/content/drive/MyDrive/Tesi/World Energy Consumption.csv")

df=df_original.copy()

In [ ]:

# Visualizzazione delle prime righe, informazioni generali e valori mancanti
df_info = df.info()
df_head = df.head()
df_missing = df.isnull().sum()

df_info, df_head, df_missing


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22012 entries, 0 to 22011
Columns: 129 entries, country to wind_share_energy
dtypes: float64(126), int64(1), object(2)
memory usage: 21.7+ MB


(None,
          country  year iso_code  population  gdp  biofuel_cons_change_pct  \
 0  ASEAN (Ember)  2000      NaN         NaN  NaN                      NaN   
 1  ASEAN (Ember)  2001      NaN         NaN  NaN                      NaN   
 2  ASEAN (Ember)  2002      NaN         NaN  NaN                      NaN   
 3  ASEAN (Ember)  2003      NaN         NaN  NaN                      NaN   
 4  ASEAN (Ember)  2004      NaN         NaN  NaN                      NaN   
 
    biofuel_cons_change_twh  biofuel_cons_per_capita  biofuel_consumption  \
 0                      NaN                      NaN                  NaN   
 1                      NaN                      NaN                  NaN   
 2                      NaN                      NaN                  NaN   
 3                      NaN                      NaN                  NaN   
 4                      NaN                      NaN                  NaN   
 
    biofuel_elec_per_capita  ...  solar_share_elec  solar_s

 **Riepilogo del dataset:**

Numero di righe: 22.012

Numero di colonne: 129

Tipi di dati: 126 colonne numeriche (float64), 1 colonna intera (int64) e 2 colonne testuali (object)

Principali colonne: country, year, iso_code, population, gdp, e varie misure di consumo energetico.

Dati mancanti: Alcune colonne hanno migliaia di valori mancanti, ad esempio:
iso_code ha 5500 valori mancanti.

population ha 3889 valori mancanti.

gdp ha 10.899 valori mancanti.

Molte colonne legate a energie rinnovabili hanno un alto numero di dati mancanti.

### Identificativi univoci

306 valori unici in country, ma solo 219 in iso_code → Alcuni paesi non hanno un codice ISO associato (es. aggregazioni regionali).

In [ ]:
unique_countries = df["country"].nunique()
unique_iso_codes = df["iso_code"].nunique() #iso_code è vuoto per i paesi aggregati -> continenti (Asia), aggregazioni (Nord America)

In [ ]:
unique_countries

306

In [ ]:
unique_iso_codes

219

Creo un codice univoco unendo anno e country

In [ ]:
df['year_country'] = df['year'].astype(str) + '_' + df['country']
print(df[['year', 'country', 'year_country']].head())

   year        country        year_country
0  2000  ASEAN (Ember)  2000_ASEAN (Ember)
1  2001  ASEAN (Ember)  2001_ASEAN (Ember)
2  2002  ASEAN (Ember)  2002_ASEAN (Ember)
3  2003  ASEAN (Ember)  2003_ASEAN (Ember)
4  2004  ASEAN (Ember)  2004_ASEAN (Ember)


### Distribuzione temporale

Dati dal 1900 al 2022.
Pochi dati per gli anni più vecchi (solo 118 voci per il 1900-1904).
Maggior numero di dati dal 2000 in poi.


In [ ]:

year_distribution = df["year"].value_counts().sort_index()
year_distribution


,count
year,
1900,118
1901,118
1902,118
1903,118
1904,118
...,...
2018,286
2019,286
2020,285


Considero solo dati dal 2000 in poi per garantire una maggiore completezza

In [ ]:
#df = df[df["year"] >= 2000]


### Check duplicati

In [ ]:
duplicate_rows = df.duplicated().sum()


### Percentuale di valori mancanti per colonna

Molte colonne hanno oltre il 50% di dati mancanti, in particolare nelle fonti di energia rinnovabile.

Colonne con oltre il 90% di valori mancanti:


*   biofuel_cons_change_pct (92%)
*   biofuel_cons_per_capita (89%)
*   wind_energy_per_capita (81%)


In [ ]:
missing_percentage = (df.isnull().sum() / len(df)) * 100
high_missing_cols = missing_percentage[missing_percentage > 50]  # Colonne con più del 50% di NaN

In [ ]:
high_missing_cols

,0
biofuel_cons_change_pct,92.063420
biofuel_cons_change_twh,87.793022
biofuel_cons_per_capita,89.542068
biofuel_consumption,87.429584
biofuel_elec_per_capita,76.035799
...,...
wind_elec_per_capita,67.903871
wind_electricity,63.674359
wind_energy_per_capita,81.532800
wind_share_elec,68.717063


Elimino colonne con più del 50% di valori mancanti.

Per le altre colonne numeriche, riempio i NaN con la mediana del paese corrispondente (se possibile) o della colonna.

In [ ]:
nulls=df.isna().sum()
nulls

,0
country,0
year,0
iso_code,5500
population,3889
gdp,10899
...,...
wind_electricity,14016
wind_energy_per_capita,17947
wind_share_elec,15126
wind_share_energy,17911


In [ ]:
#Per elementi come il PIL non ha senso sostituire i valori nulli con uno 0, quindi uso ffil e poi bfill
#FWD = Sostituisce i valori NaN (mancanti) con l'ultimo valore disponibile precedente nella stessa colonna.
nulls=df.isna().sum()
for i in nulls.index:
    if nulls[i]>0:
        df[i].ffill(inplace=True)
        df[i].bfill(inplace=True)
df.isna().sum()

<ipython-input-15-bbd9c5bd1da1>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].ffill(inplace=True)
<ipython-input-15-bbd9c5bd1da1>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: valu

,0
country,0
year,0
iso_code,0
population,0
gdp,0
...,...
wind_electricity,0
wind_energy_per_capita,0
wind_share_elec,0
wind_share_energy,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22012 entries, 0 to 22011
Columns: 130 entries, country to year_country
dtypes: float64(126), int64(1), object(3)
memory usage: 21.8+ MB


### Distribuzione statistica delle colonne numeriche

La popolazione varia da 1.833 abitanti a 7,97 miliardi.

Il PIL oscilla tra 164 milioni e 113.6 trilioni di dollari.

Alcune metriche (es. percentuali di crescita delle rinnovabili) hanno valori massimi molto elevati, suggerendo la presenza di outlier.

In [ ]:
numeric_summary = df.describe().T
numeric_summary


,count,mean,std,min,25%,50%,75%,max
year,22012.0,1.974213e+03,3.505865e+01,1.900000e+03,1.946000e+03,1.984000e+03,2.003000e+03,2.022000e+03
population,22012.0,1.681924e+08,6.548669e+08,1.833000e+03,1.914930e+06,6.982328e+06,3.121812e+07,7.975105e+09
gdp,22012.0,7.174825e+11,3.696685e+12,1.642060e+08,2.070923e+10,7.661752e+10,3.374119e+11,1.136302e+14
biofuel_cons_change_pct,22012.0,8.854751e+00,7.977120e+01,-1.000000e+02,-6.310000e-01,4.335000e+00,1.027200e+01,5.659328e+03
biofuel_cons_change_twh,22012.0,4.254203e+00,1.114127e+01,-5.084300e+01,-1.100000e-02,9.200000e-02,1.246000e+00,1.411310e+02
...,...,...,...,...,...,...,...,...
wind_elec_per_capita,22012.0,2.518346e+02,6.421619e+02,0.000000e+00,0.000000e+00,5.000000e-03,1.160290e+02,3.219852e+03
wind_electricity,22012.0,8.543793e+01,2.623963e+02,0.000000e+00,0.000000e+00,1.000000e-02,2.749000e+00,2.139230e+03
wind_energy_per_capita,22012.0,1.192761e+03,1.916768e+03,0.000000e+00,4.586750e+00,3.401140e+02,1.487169e+03,8.422012e+03
wind_share_elec,22012.0,3.430737e+00,6.458429e+00,0.000000e+00,0.000000e+00,7.300000e-02,4.537000e+00,5.684000e+01


### Selezione variabili chiave (features)

Mantengo colonne essenziali (country, year, iso_code, population, gdp, fonti di energia).

Rimuovo colonne ridondanti.

In [ ]:
#columns_to_keep = ["country", "year", "iso_code", "population", "gdp"] #-> ddp=PIL

In [ ]:
#energy_columns = [col for col in df.columns if "energy" in col or "electricity" in col or "consumption" in col]
#columns_to_keep.extend(energy_columns)
#df = df[columns_to_keep]

### Feature Engineering

In [ ]:
df_ = df[['iso_code',
           'country',
           'year',
          #'year_country',
          #'gdp',
           'biofuel_electricity',
           'hydro_electricity',
           'nuclear_electricity',
           'solar_electricity',
           'wind_electricity',
           'other_renewable_electricity',
           'coal_electricity',
           'gas_electricity',
           'oil_electricity',
           'biofuel_elec_per_capita',
           'hydro_elec_per_capita',
           'nuclear_elec_per_capita',
           'solar_elec_per_capita',
           'wind_elec_per_capita',
           'coal_elec_per_capita',
           'gas_elec_per_capita',
           'oil_elec_per_capita',
           'population'
          ]]

df_

,iso_code,country,year,biofuel_electricity,hydro_electricity,nuclear_electricity,solar_electricity,wind_electricity,other_renewable_electricity,coal_electricity,...,oil_electricity,biofuel_elec_per_capita,hydro_elec_per_capita,nuclear_elec_per_capita,solar_elec_per_capita,wind_elec_per_capita,coal_elec_per_capita,gas_elec_per_capita,oil_elec_per_capita,population
0,AFG,ASEAN (Ember),2000,5.60,50.37,0.0,0.00,0.0,22.53,71.03,...,61.50,0.000,15.862,0.0,0.000,0.0,0.000,0.0,8.187,4707744.0
1,AFG,ASEAN (Ember),2001,6.02,54.26,0.0,0.00,0.0,22.42,80.02,...,55.14,0.000,15.862,0.0,0.000,0.0,0.000,0.0,8.187,4707744.0
2,AFG,ASEAN (Ember),2002,6.06,53.32,0.0,0.00,0.0,22.67,87.16,...,56.15,0.000,15.862,0.0,0.000,0.0,0.000,0.0,8.187,4707744.0
3,AFG,ASEAN (Ember),2003,6.69,53.28,0.0,0.00,0.0,22.43,98.51,...,56.01,0.000,15.862,0.0,0.000,0.0,0.000,0.0,8.187,4707744.0
4,AFG,ASEAN (Ember),2004,7.57,52.88,0.0,0.00,0.0,24.14,111.62,...,58.33,0.000,15.862,0.0,0.000,0.0,0.000,0.0,8.187,4707744.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22007,ZWE,Zimbabwe,2018,0.39,5.05,0.0,0.02,0.0,0.39,3.69,...,0.04,25.910,335.499,0.0,1.329,0.0,245.147,0.0,2.657,15052191.0
22008,ZWE,Zimbabwe,2019,0.38,4.17,0.0,0.03,0.0,0.38,3.62,...,0.04,24.748,271.580,0.0,1.954,0.0,235.760,0.0,2.605,15354606.0
22009,ZWE,Zimbabwe,2020,0.35,3.81,0.0,0.03,0.0,0.35,3.36,...,0.04,22.336,243.145,0.0,1.915,0.0,214.427,0.0,2.553,15669663.0
22010,ZWE,Zimbabwe,2021,0.38,4.00,0.0,0.04,0.0,0.38,3.57,...,0.04,23.760,250.101,0.0,2.501,0.0,223.215,0.0,2.501,15993525.0


scelgo l'Italia come "paese test" per le trasformazioni

In [ ]:
df_.loc[df_["country"] == "Europe", "iso_code"] = "EUR"
df_.loc[df_["country"] == "ASEAN (Ember)", "iso_code"] = "ASE"




In [ ]:
df_.loc[(df_['country'] == 'Italy') & (df['year'] == 2020)]


,iso_code,country,year,biofuel_electricity,hydro_electricity,nuclear_electricity,solar_electricity,wind_electricity,other_renewable_electricity,coal_electricity,...,oil_electricity,biofuel_elec_per_capita,hydro_elec_per_capita,nuclear_elec_per_capita,solar_elec_per_capita,wind_elec_per_capita,coal_elec_per_capita,gas_elec_per_capita,oil_elec_per_capita,population
10024,ITA,Italy,2020,19.62,47.55,0.0,24.94,18.76,25.65,13.38,...,14.09,329.745,799.152,0.0,419.156,315.291,224.872,2247.037,236.804,59500576.0


### Controllo coerenza dati

In [ ]:
# - Controllo se il totale energia è la somma delle fonti
energy_sources = [col for col in df.columns if "consumption" in col and col != "primary_energy_consumption"]
df["energy_check"] = df[energy_sources].sum(axis=1)
df["energy_diff"] = np.abs(df["energy_check"] - df["primary_energy_consumption"])

# - Controlliamo se ci sono duplicati per country-year
duplicates = df[df.duplicated(subset=["country", "year"], keep=False)]

# Rimuoviamo colonne di controllo temporanee
df = df.drop(columns=["energy_check", "energy_diff"], errors="ignore")

# Output finale
df.shape, duplicates.shape


((22012, 130), (0, 132))

### Aggregazione energia

In [ ]:
df = df_.melt(id_vars=['iso_code', 'country', 'year', 'population'],
              var_name='fuel',
              value_name='production'
)

df

,iso_code,country,year,population,fuel,production
0,ASE,ASEAN (Ember),2000,4707744.0,biofuel_electricity,5.600
1,ASE,ASEAN (Ember),2001,4707744.0,biofuel_electricity,6.020
2,ASE,ASEAN (Ember),2002,4707744.0,biofuel_electricity,6.060
3,ASE,ASEAN (Ember),2003,4707744.0,biofuel_electricity,6.690
4,ASE,ASEAN (Ember),2004,4707744.0,biofuel_electricity,7.570
...,...,...,...,...,...,...
374199,ZWE,Zimbabwe,2018,15052191.0,oil_elec_per_capita,2.657
374200,ZWE,Zimbabwe,2019,15354606.0,oil_elec_per_capita,2.605
374201,ZWE,Zimbabwe,2020,15669663.0,oil_elec_per_capita,2.553
374202,ZWE,Zimbabwe,2021,15993525.0,oil_elec_per_capita,2.501


In [ ]:
df.loc[(df['country'] == 'Italy') & (df['year'] == 2020)]


,iso_code,country,year,population,fuel,production
10024,ITA,Italy,2020,59500576.0,biofuel_electricity,19.620
32036,ITA,Italy,2020,59500576.0,hydro_electricity,47.550
54048,ITA,Italy,2020,59500576.0,nuclear_electricity,0.000
76060,ITA,Italy,2020,59500576.0,solar_electricity,24.940
98072,ITA,Italy,2020,59500576.0,wind_electricity,18.760
120084,ITA,Italy,2020,59500576.0,other_renewable_electricity,25.650
142096,ITA,Italy,2020,59500576.0,coal_electricity,13.380
164108,ITA,Italy,2020,59500576.0,gas_electricity,133.700
186120,ITA,Italy,2020,59500576.0,oil_electricity,14.090
208132,ITA,Italy,2020,59500576.0,biofuel_elec_per_capita,329.745


In [ ]:
#Creo la colonna 'pro_capita' per spostare i valori di fuel_pro_capita

def split_type(row):
    if row['fuel'].find('per_capita') != -1:
        value = row['production']
    else:
        value = 0
    return value

df['per_capita'] = df.apply(split_type, axis=1)

df

,iso_code,country,year,population,fuel,production,per_capita
0,ASE,ASEAN (Ember),2000,4707744.0,biofuel_electricity,5.600,0.000
1,ASE,ASEAN (Ember),2001,4707744.0,biofuel_electricity,6.020,0.000
2,ASE,ASEAN (Ember),2002,4707744.0,biofuel_electricity,6.060,0.000
3,ASE,ASEAN (Ember),2003,4707744.0,biofuel_electricity,6.690,0.000
4,ASE,ASEAN (Ember),2004,4707744.0,biofuel_electricity,7.570,0.000
...,...,...,...,...,...,...,...
374199,ZWE,Zimbabwe,2018,15052191.0,oil_elec_per_capita,2.657,2.657
374200,ZWE,Zimbabwe,2019,15354606.0,oil_elec_per_capita,2.605,2.605
374201,ZWE,Zimbabwe,2020,15669663.0,oil_elec_per_capita,2.553,2.553
374202,ZWE,Zimbabwe,2021,15993525.0,oil_elec_per_capita,2.501,2.501


In [ ]:
df.loc[(df['country'] == 'Italy') & (df['year'] == 2020)]


,iso_code,country,year,population,fuel,production,per_capita
10024,ITA,Italy,2020,59500576.0,biofuel_electricity,19.620,0.000
32036,ITA,Italy,2020,59500576.0,hydro_electricity,47.550,0.000
54048,ITA,Italy,2020,59500576.0,nuclear_electricity,0.000,0.000
76060,ITA,Italy,2020,59500576.0,solar_electricity,24.940,0.000
98072,ITA,Italy,2020,59500576.0,wind_electricity,18.760,0.000
120084,ITA,Italy,2020,59500576.0,other_renewable_electricity,25.650,0.000
142096,ITA,Italy,2020,59500576.0,coal_electricity,13.380,0.000
164108,ITA,Italy,2020,59500576.0,gas_electricity,133.700,0.000
186120,ITA,Italy,2020,59500576.0,oil_electricity,14.090,0.000
208132,ITA,Italy,2020,59500576.0,biofuel_elec_per_capita,329.745,329.745


In [ ]:
def clean_production(row):
    if row['per_capita'] != 0:
        value = 0
    else:
        value = row['production']
    return value


df['production'] = df.apply(clean_production, axis=1)

df

,iso_code,country,year,population,fuel,production,per_capita
0,ASE,ASEAN (Ember),2000,4707744.0,biofuel_electricity,5.60,0.000
1,ASE,ASEAN (Ember),2001,4707744.0,biofuel_electricity,6.02,0.000
2,ASE,ASEAN (Ember),2002,4707744.0,biofuel_electricity,6.06,0.000
3,ASE,ASEAN (Ember),2003,4707744.0,biofuel_electricity,6.69,0.000
4,ASE,ASEAN (Ember),2004,4707744.0,biofuel_electricity,7.57,0.000
...,...,...,...,...,...,...,...
374199,ZWE,Zimbabwe,2018,15052191.0,oil_elec_per_capita,0.00,2.657
374200,ZWE,Zimbabwe,2019,15354606.0,oil_elec_per_capita,0.00,2.605
374201,ZWE,Zimbabwe,2020,15669663.0,oil_elec_per_capita,0.00,2.553
374202,ZWE,Zimbabwe,2021,15993525.0,oil_elec_per_capita,0.00,2.501


In [ ]:
df['fuel'] = df['fuel'].str.replace('_elec_per_capita', '')
df['fuel'] = df['fuel'].str.replace('_electricity', '')

df['fuel'].unique()

array(['biofuel', 'hydro', 'nuclear', 'solar', 'wind', 'other_renewable',
       'coal', 'gas', 'oil'], dtype=object)

In [ ]:
#visiono i primi 50 paesi per numerosità di features (analisi specifica in notebook Main)
top_50_series = df.groupby(df['country']).count().sum(axis = 1).sort_values(ascending = False).head(50)
top_50_list = df.groupby(df['country']).count().sum(axis = 1).sort_values(ascending = False).head(50).index.tolist()
top_50_series

,0
country,
Zimbabwe,12546
South Korea,12546
India,12546
Romania,12546
Hungary,12546
High-income countries,12546
Saudi Arabia,12546
Greece,12546
Germany,12546


In [ ]:
df = df.drop(df[df['country'] == 'World'].index)
#df = df[~df["country"].isin(["valore1", "valore2", "valore3"])]

df['country'].unique()

array(['ASEAN (Ember)', 'Afghanistan', 'Africa', 'Africa (EI)',
       'Africa (Ember)', 'Africa (Shift)', 'Albania', 'Algeria',
       'American Samoa', 'Angola', 'Antarctica', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia & Oceania (EIA)',
       'Asia (Ember)', 'Asia Pacific (EI)', 'Asia and Oceania (Shift)',
       'Australia', 'Australia and New Zealand (EIA)', 'Austria',
       'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'CIS (EI)', 'Cambodia', 'Cameroon', 'Canada',
       'Cape Verde', 'Cayman Islands', 'Central & South America (EIA)',
       'Central African Republic', 'Central America (EI)',
       'Central and South America (Shift)', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo', 'Cook Isla

In [ ]:
df_final = df.groupby(['iso_code', 'country', 'year', 'population', 'fuel']).sum().reset_index()

df_final.loc[(df_final['country'] == 'Italy') & (df_final['year'] == 2020)]

,iso_code,country,year,population,fuel,production,per_capita
94338,ITA,Italy,2020,59500576.0,biofuel,19.62,329.745
94339,ITA,Italy,2020,59500576.0,coal,13.38,224.872
94340,ITA,Italy,2020,59500576.0,gas,133.70,2247.037
94341,ITA,Italy,2020,59500576.0,hydro,47.55,799.152
94342,ITA,Italy,2020,59500576.0,nuclear,0.00,0.000
94343,ITA,Italy,2020,59500576.0,oil,14.09,236.804
94344,ITA,Italy,2020,59500576.0,other_renewable,25.65,0.000
94345,ITA,Italy,2020,59500576.0,solar,24.94,419.156
94346,ITA,Italy,2020,59500576.0,wind,18.76,315.291


In [ ]:
df_final.to_csv('/content/drive/MyDrive/Tesi/WorldConsumption_Prepdataset.csv',index=False)


Per il momento mantengo tutti i paesi, proseguendo nelle analisi, decido se utilizzare solo un numero limitati, eventualmente un focus su Europa

Creo un dataset di descrizioni utili per l'app come fonte Enciclopedia Italiana

In [ ]:
description = {
    'fuel': ['biofuel',
            'coal',
            'gas',
            'hydro',
            'nuclear',
            'oil',
            'other_renewable',
            'solar',
            'wind'],


    'description': ['Carburanti ottenuti da materie prime di origine agricola. A differenza dei carburanti tradizionali, che derivano da combustibili fossili, hanno il vantaggio di provenire da materie prime rinnovabili e ubiquitarie. I principali tipi di b. sono l’alcol etilico ottenuto per via fermentativa e il biodiesel. Interesse per l’autotrazione ha anche il biogas',
                   'Uno dei più importanti combustibili fossili primari, un materiale solido ricco di carbonio, di solito marrone o nero, che si trova più frequentemente in depositi sedimentari stratificati. Il carbone è definito come una sostanza con più del 50% in peso (o il 70% in volume) di materia carboniosa, prodotta dalla compattazione e dallindurimento di resti vegetali alterati, in particolare depositi di torba',
                   'Chiamato anche gas metano o metano naturale, è un idrocarburo gassoso incolore e altamente infiammabile, composto principalmente da metano ed etano. È un tipo di petrolio che si trova comunemente in associazione con il greggio.',
                   'Elettricità prodotta da generatori azionati da turbine che convertono lenergia potenziale dellacqua che cade o scorre rapidamente in energia meccanica. ',
                   'Elettricità generata da centrali elettriche che traggono calore dalla fissione in un reattore nucleare. A parte il reattore, che svolge il ruolo di caldaia in una centrale a combustibili fossili, una centrale nucleare è simile a una grande centrale a carbone, con pompe, valvole, generatori di vapore, turbine, generatori elettrici, condensatori e attrezzature associate.',
                   'Chiamato anche olio da forno, è un combustibile costituito principalmente da residui della distillazione del petrolio greggio. Viene utilizzato principalmente per caldaie a vapore nelle centrali elettriche, a bordo delle navi e negli impianti industriali. ',
                   'Elettricità generata da altre forme di fonti di energia rinnovabile',
                   'Forma di energia rinnovabile generata dalla conversione della luce solare e della luce artificiale in elettricità.',
                   'Forma di conversione dellenergia in cui le turbine trasformano lenergia cinetica del vento in energia meccanica o elettrica utilizzabile per la produzione di energia.']}

df_desc = pd.DataFrame(description)

df_desc

,fuel,description
0,biofuel,Carburanti ottenuti da materie prime di origin...
1,coal,Uno dei più importanti combustibili fossili pr...
2,gas,"Chiamato anche gas metano o metano naturale, è..."
3,hydro,Elettricità prodotta da generatori azionati da...
4,nuclear,Elettricità generata da centrali elettriche ch...
5,oil,"Chiamato anche olio da forno, è un combustibil..."
6,other_renewable,Elettricità generata da altre forme di fonti d...
7,solar,Forma di energia rinnovabile generata dalla co...
8,wind,Forma di conversione dellenergia in cui le tur...


In [ ]:
df_desc.to_csv('/content/drive/MyDrive/Tesi/EnerdyDecription.csv',index=False)


In [ ]:
df_original= pd.read_csv("/content/drive/MyDrive/Tesi/WorldConsumption_Prepdataset.csv")
df_original

,iso_code,country,year,population,fuel,production,per_capita
0,ABW,Aruba,1986,64570.0,biofuel,0.00,0.000
1,ABW,Aruba,1986,64570.0,coal,0.00,0.000
2,ABW,Aruba,1986,64570.0,gas,3.18,1139.388
3,ABW,Aruba,1986,64570.0,hydro,2.20,788.256
4,ABW,Aruba,1986,64570.0,nuclear,1.85,662.852
...,...,...,...,...,...,...,...
196996,ZWE,Zimbabwe,2022,16320539.0,nuclear,0.00,0.000
196997,ZWE,Zimbabwe,2022,16320539.0,oil,0.04,2.501
196998,ZWE,Zimbabwe,2022,16320539.0,other_renewable,0.38,0.000
196999,ZWE,Zimbabwe,2022,16320539.0,solar,0.04,2.501
